# Projet DM

In [37]:
!pip install unidecode
!pip install spacy
!python -m spacy download fr_core_news_sm

     ---------------------------------------- 0.0/16.3 MB ? eta -:--:--
     --------- ------------------------------ 3.7/16.3 MB 27.3 MB/s eta 0:00:01
     --------------------------- ----------- 11.3/16.3 MB 32.1 MB/s eta 0:00:01
     ---------------------------------------- 16.3/16.3 MB 31.1 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [38]:
import json
import os
import pandas as pd
import re
from unidecode import unidecode


def load_jsonl(filename):
    """Load a .jsonl file. Tries both the given path and the 'data' folder.
    Returns a list of dicts."""
    p = os.path.join('data', filename)
    if os.path.exists(p):
        data = []
        with open(p, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                try:
                    data.append(json.loads(line))
                except Exception as e:
                    print(f"Error parsing a line in {p}: {e}")
        return data
    raise FileNotFoundError(f"File not found: {filename} (tried: {p})")

### 1. Data Upload

In [39]:
# Load the data 
train_path = 'train_v2.jsonl'
test_path = 'test_v4.jsonl'

train_data = load_jsonl(train_path)
test_data = load_jsonl(test_path)

print(f"Train records: {len(train_data)}, Test records: {len(test_data)}")

# Convert to DataFrame for easier handling
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Quick preview
print("-- Train head --")
print(train_df.head())
print("\n-- Info --")
print(train_df.info())

Train records: 492, Test records: 519
-- Train head --
                                                text acronym  \
0  LRA  limite de résistance des attelages PAR po...     PAR   
1                              Désigna -tion des PN       PN   
2  prédéterminées de trains : _x0001_ les masses ...      EM   
3  /Commentaires N° AC B81500 thermique:  compati...      AC   
4  kilomètres/heure (ex : 12 pour 120 km/h), _x00...     TIV   

                                             options  
0  {'Plan d'action régularité': False, 'Poste d'a...  
1  {'Passages à niveau : fichier des pn, recensem...  
2  {'EMERAINVILLE PONTAULT COMBAULT': False, 'Eng...  
3  {'ACcès': False, 'Agent d'aCcompagnement ': Fa...  
4  {'THIVIERS': False, 'Trafic international voya...  

-- Info --
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     492 non-null    object
 1 

### 2. Data Cleaning

In [40]:
# Basic cleaning and review
# 1) Columns, types, and missing values
print("Columns:", list(train_df.columns))
print("Missing values per column:\n", train_df.isna().sum())

# 2) Remove simple duplicates
train_df = train_df.drop_duplicates(subset=['text', 'acronym']).reset_index(drop=True)

# 3) Fill missing values in text columns with empty string (example)
for col in train_df.select_dtypes(include='object').columns:
    train_df[col] = train_df[col].fillna('')

print("After cleaning, shape:", train_df.shape)

Columns: ['text', 'acronym', 'options']
Missing values per column:
 text       0
acronym    0
options    0
dtype: int64
After cleaning, shape: (492, 3)


### 3. Pre-Processing

In [ ]:
import re

# Remove OCR artifacts and bad characters
def clean_ocr_artifacts(text):
    # Remove _x0001_ and similar hex patterns
    text = re.sub(r'_x[0-9a-fA-F]{4}_', '', text)
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

print("Sample text before normalization:")
print(train_df['text'].sample(3).values)
# Text normalization lowercasing
for col in ['text', 'acronym']:
    train_df[col] = train_df[col].str.lower()
train_df['options'] = train_df['options'].apply(lambda opts: {k.lower(): v for k, v in opts.items()})
# clean ocr artefacts
for col in ['text', 'acronym']:
    train_df[col] = train_df[col].apply(clean_ocr_artifacts)
train_df['options'] = train_df['options'].apply(lambda opts: {clean_ocr_artifacts(k): v for k, v in opts.items()})

#Apply unidecode to remove accents
for col in ['text', 'acronym']:
    train_df[col] = train_df[col].apply(lambda x: unidecode(x).encode('latin1').decode('latin1'))
train_df['options'] = train_df['options'].apply(lambda opts: {unidecode(k).encode('latin1').decode('latin1'): v for k, v in opts.items()})

print("After text normalization, shape:", train_df.shape)
print("Sample text after normalization:")
print(train_df['text'].sample(3).values)

Sample text before normalization:
['− Entreprises titulaires d’une attestation de sécurité et exerçant des activités relatives à la gestion de l\'infrastructure − Etablissement Infra Circulation Centre : Pôle Sécurité, COGC Centre, BHR Centre, Gares et Postes concernés   Métier "Maintenance & Travaux" '
 'CSS central sous-stations DAAT Dispositif d’arrêt automatique des trains DT double traction EAS équipement agent seul EF entreprise ferroviaire '
 "EF entreprise ferroviaire EM engin moteur EPSF établissement public de sécurité ferroviaire GID gestionnaire d'infrastructure délégué HLP haut le pied "]
After text normalization, shape: (492, 3)
Sample text after normalization:
['bif 2 lyon / 2 biszone telecommandee par le pcd de dijonfa'
 'amec autorisation de mise en exploitation commerciale dt double traction ef entreprise ferroviaire em engin moteur hlp haut le pied '
 "cclr chef circulation local regulateur centre de circulation cle consigne locale d'exploitation cogc centre operatio

In [42]:
train_df

,text,acronym,options
0,lra limite de resistance des attelages par po...,par,"{'plan d'action regularite': False, 'poste d'a..."
1,designa -tion des pn,pn,"{'passages a niveau : fichier des pn, recensem..."
2,predeterminees de trains : _x0001_ les masses ...,em,"{'emerainville pontault combault': False, 'eng..."
3,/commentaires ndeg ac b81500 thermique: compa...,ac,"{'acces': False, 'agent d'accompagnement ': Fa..."
4,"kilometres/heure (ex : 12 pour 120 km/h), _x00...",tiv,"{'thiviers': False, 'trafic international voya..."
...,...,...,...
487,teur des quais est >= 760 mm.] [ 1500v : (lyon...,us,"{'us (val-d'oise)': False, 'unite simple : mod..."
488,serie consideree sans risque de deterioration ...,rff,"{'roscoff': False, 'reseau ferre de france': T..."
489,figurant pas dans la cle propre a l'etablissem...,cle,"{'clermont ferrand': False, 'consigne locale d..."
490,km fin restrictions me120 me100 ma100 830000 l...,cmt,"{'controle des mobiles travaux': False, 'contr..."


In [43]:
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("stopwords-fr")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = stopwords.words("french")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading stopwords-fr: Package 'stopwords-fr' not
[nltk_data]     found in index


In [44]:
#delete stop_words
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)
train_df['text_clean'] = train_df['text'].apply(remove_stopwords)
print("After stopword removal, shape:", train_df.shape)

After stopword removal, shape: (492, 4)


In [45]:
# Lemmatization setup
import spacy
nlp = spacy.load("fr_core_news_sm")
# Apply lemmatization
def lemmatize_text(text):
    doc = nlp(str(text))
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

train_df['text_clean'] = train_df['text'].apply(lemmatize_text)
print("After lemmatization:")
print(train_df['text'].head(5).values)
print(train_df['text_clean'].head(5).values)


After lemmatization:
["lra  limite de resistance des attelages par poste d'aiguillage et de regulation pl pleine ligne pn passage a niveau rfn reseau ferre national "
 '  designa -tion des pn '
 "predeterminees de trains : _x0001_ les masses admissibles standard pour les trains (et eventuellement celles pour les trains entiers) pour chaque serie ou groupement de series d'em autorises a circuler sur tout ou partie du domaine couvert par le document, _x0001_ la limite de resistance des attelages standard [en distinguant eventuellement avec "
 '/commentaires ndeg ac b81500 thermique:  compatibilite : oui/restrictions : les agc sont interdits a la desserte commerciale des etablis-sements dont la hauteur des quais est >= 760 mm '
 "kilometres/heure (ex : 12 pour 120 km/h), _x0001_ d'une lettre indiquant le respect ou non des tableaux indicateurs de vitesse de type c (c respecte les tiv, n ne respecte pas les tiv). engin moteur vehicule ayant la capacite de se deplacer par ses propres moyens